In [17]:
import pandas as pd
import numpy as np
import gc
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [5]:
df_train = pd.read_csv("../../data/application_train.csv")
df_test = pd.read_csv("../../data/application_test.csv")
TARGET = df_train['TARGET']


In [6]:
test_id = df_test.iloc[:, 0]
train_id = df_train.iloc[:, 0]

In [7]:
def get_type(x):
    if isinstance(x[0], str):
        return 0
    if len( np.unique(x[:400])) < 15:
        return 0
    else:
        return 1
    
def plot(x):
    global TARGET
    _type = get_type(x)
    if _type == 0:
        discrete_plot(x, TARGET)
    else:
        continuous_plot(x, TARGET)
    
def discrete_plot(x, y):
    temp = pd.concat((x, y), axis=1)
    temp['value'] = 1
    temp = temp.dropna()
    temp.groupby(list(temp.columns)[:2]).sum().unstack().plot(kind='bar')
    print(temp.groupby([temp.columns[0]])['TARGET'].mean())
    print(temp.groupby([temp.columns[0]])['TARGET'].count())

def continuous_plot(x, y):
    x = x.dropna()
    sns.distplot(x.loc[y==0])
    sns.distplot(x.loc[y==1])


In [8]:
y_train = df_train['TARGET']
df_train = df_train.iloc[:, 2:]
df_test = df_test.iloc[:, 1:]

In [9]:
ext_1_train = df_train['EXT_SOURCE_1']
ext_1_test = df_test['EXT_SOURCE_1']

In [10]:
feature_by_missing = df_train.columns[df_train.isnull().sum()/df_train.shape[0] > 0.5]

### 添加 [kernel](https://www.kaggle.com/poohtls/fork-of-fork-lightgbm-with-simple-features/code) 中的特征

In [11]:
len_train = df_train.shape[0]
len_test = df_test.shape[0]

In [12]:
df = df_train.append(df_test).reset_index()

In [13]:
docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
live = [_f for _f in df.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]

In [14]:
# A lot of the continuous days variables have integers as missing value indicators.
def mean_std(x):
    return (x - np.mean(x)) / np.std(x)

df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

inc_by_org = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']
df['NEW_CREDIT_TO_GOODS_RATIO'] = mean_std(df['AMT_CREDIT'] / df['AMT_GOODS_PRICE'])
df['NEW_DOC_IND_KURT'] = mean_std(df[docs].kurtosis(axis=1))
df['NEW_LIVE_IND_SUM'] = mean_std(df[live].sum(axis=1))
df['NEW_INC_PER_CHLD'] = mean_std(df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN']))
df['NEW_INC_BY_ORG'] = mean_std(df['ORGANIZATION_TYPE'].map(inc_by_org))
df['NEW_EMPLOY_TO_BIRTH_RATIO'] = mean_std(df['DAYS_EMPLOYED'] / df['DAYS_BIRTH'])
df['NEW_ANNUITY_TO_INCOME_RATIO'] = mean_std(df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL']))
df['NEW_SOURCES_PROD'] = mean_std(df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3'])
df['NEW_EXT_SOURCES_MEAN'] = mean_std(df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1))
df['NEW_SCORES_STD'] = mean_std(df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1))
df['NEW_SCORES_STD'] = mean_std(df['NEW_SCORES_STD'].fillna(df['NEW_SCORES_STD'].mean()))
df['NEW_CAR_TO_BIRTH_RATIO'] = mean_std(df['OWN_CAR_AGE'] / df['DAYS_BIRTH'])
df['NEW_CAR_TO_EMPLOY_RATIO'] = mean_std(df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED'])
df['NEW_PHONE_TO_BIRTH_RATIO'] = mean_std(df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH'])
df['NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER'] = mean_std(df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED'])
df['NEW_CREDIT_TO_INCOME_RATIO'] = mean_std(df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL'])
df['LOAN_INCOME_RATIO'] = mean_std(df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL'])
df['ANNUITY_INCOME_RATIO'] = mean_std(df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL'])

# Number of overall payments (I think!)
df['ANNUITY LENGTH'] = mean_std(df['AMT_CREDIT'] / df['AMT_ANNUITY'])

# Social features
df['WORKING_LIFE_RATIO'] = mean_std(df['DAYS_EMPLOYED'] / df['DAYS_BIRTH'])
df['INCOME_PER_FAM'] = mean_std(df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS'])
df['CHILDREN_RATIO'] = mean_std(df['CNT_CHILDREN'] / df['CNT_FAM_MEMBERS'])

In [15]:
dropcolum=['index']
df= df.drop(dropcolum,axis=1)

In [16]:
df_train = df.iloc[: len_train]
df_test = df.iloc[len_train: , ]

In [18]:
del df
gc.collect()

219

In [19]:
df_train = df_train[[i for i in df_train.columns if i not in feature_by_missing]]
df_test = df_test[[i for i in df_train.columns if i not in feature_by_missing]]

In [20]:
columns = df_train.columns
drop_num  = []
select_num = []

# 下面是对特征进行处理，跳转到最后进行特征提取并且单独测试

In [398]:
df_train.iloc[:, 4] = list(map(lambda x: 5 if x >= 5 else x, df_train.iloc[:, 4])) 
df_test.iloc[:, 4] = list(map(lambda x: 5 if x >= 5 else x, df_test.iloc[:, 4]))

In [399]:
def replace(x):
    s = 50000
    for i in range(12):
        if x < s + 25000 * i:
            return str(i)
    return '12'

df_train.iloc[:, 5] = list(map(lambda x:  replace(x), df_train.iloc[:, 5]))
df_test.iloc[:, 5] = list(map(lambda x:  replace(x), df_test.iloc[:, 5]))

In [400]:
def replace(x):
    s = 600000
    for i in range(10):
        if x < s + 100000 * i:
            return str(i)
    return '10'

df_train.iloc[:, 6] = list(map(lambda x:  replace(x) if not pd.isnull(x) else x, df_train.iloc[:, 6]))
df_test.iloc[:, 6] = list(map(lambda x:  replace(x) if not pd.isnull(x) else x, df_test.iloc[:, 6]))

In [401]:

df_test.iloc[:, 7] = mean_std(np.log(df_test.iloc[:, 7]))
df_train.iloc[:, 7] = mean_std(np.log(df_train.iloc[:, 7]))

In [402]:
def replace(x):
    s = 400000
    for i in range(4):
        if x < s + 200000 * i:
            return str(i)
    return '4'

df_train.iloc[:, 8] = list(map(lambda x:  replace(x) if not pd.isnull(x) else x, df_train.iloc[:, 8]))
df_test.iloc[:, 8] = list(map(lambda x:  replace(x) if not pd.isnull(x) else x, df_test.iloc[:, 8]))
# plot(df_train.iloc[:, 8])

In [403]:
map_dic = {
    'Children' : '1',
    'Family' : '1',
    'Spouse, partner': '1',
    'Unaccompanied' : '0',
    'Group of people': '2',
    'Other_A': '2',
    'Other_B': '2',
}

df_train.iloc[:, 9] = list(map(lambda x: map_dic[x] if not pd.isnull(x) else x, df_train.iloc[:, 9]))
df_test.iloc[:, 9] = list(map(lambda x: map_dic[x] if not pd.isnull(x) else x, df_test.iloc[:, 9]))

In [404]:
map_dic = {
    'Businessman':'bus_and_stu', 'Student':'bus_and_stu', 
    'Maternity leave': 'mat_and_une', 'Unemployed':'mat_and_une'
}

df_train.iloc[:, 10] = list(map(lambda x: map_dic[x] if x in map_dic else x, df_train.iloc[:, 10]))
df_test.iloc[:, 10] = list(map(lambda x: map_dic[x] if x in map_dic else x, df_test.iloc[:, 10]))

In [405]:
df_train.iloc[:, 14] = pd.Series(list(map(lambda x: 1 if x >= 0.03 else 0 if not pd.isnull(x) else x, df_train.iloc[:, 14])))
df_test.iloc[:, 14] = pd.Series(list(map(lambda x: 1 if x >= 0.03 else 0 if not pd.isnull(x) else x, df_test.iloc[:, 14])))

In [406]:
## 重要特征！
df_train['days_birth_dummy'] = list(map(lambda x: 1 if x > -15000 else 0 if not pd.isnull(x) else x, df_train.iloc[:, 15]))
df_test['days_birth_dummy'] = list(map(lambda x: 1 if x > -15000 else 0 if not pd.isnull(x) else x, df_test.iloc[:, 15]))
df_train.iloc[:, 15] = mean_std(df_train.iloc[:, 15])
df_test.iloc[:, 15] = mean_std(df_test.iloc[:, 15])

In [407]:
df_train['days_employed_dummy'] = df_train.iloc[:, 16].map(lambda x: '2' if x>0 else '1' if x > -2100 else '0' if not pd.isnull(x) else x)
df_test['days_employed_dummy'] = df_test.iloc[:, 16].map(lambda x: '2' if x>0 else '1' if x > -2100 else '0' if not pd.isnull(x) else x)


df_test.iloc[:, 16] = mean_std(df_test.iloc[:, 16].map(lambda x: np.nan if x>0 else x))
df_train.iloc[:, 16] = mean_std(df_train.iloc[:, 16].map(lambda x: np.nan if x>0 else x))


In [408]:
df_train['days_registration_dummy'] = df_train.iloc[:, 17].map(lambda x: 1 if x> -6000 else 0 if not pd.isnull(x) else x)
df_test['days_registration_dummy'] = df_test.iloc[:, 17].map(lambda x: 1 if x>-6000 else 0 if not pd.isnull(x) else x)

df_test.iloc[:, 17] = mean_std(df_test.iloc[:, 17])
df_train.iloc[:, 17] = mean_std(df_train.iloc[:, 17])

In [409]:
df_train['days_idpub_dummy'] = df_train.iloc[:, 18].map(lambda x: 1 if x> -3100 else 0 if not pd.isnull(x) else x)
df_test['days_idpub_dummy'] = df_test.iloc[:, 18].map(lambda x: 1 if x>-3100 else 0 if not pd.isnull(x) else x)

df_test.iloc[:, 18] = mean_std(df_test.iloc[:, 18])
df_train.iloc[:, 18] = mean_std(df_train.iloc[:, 18])

In [410]:
#df_train = df_train.drop(columns=columns[19])
#df_test = df_test.drop(columns=columns[19])
drop_num.append(19)

In [411]:
drop_num.append(22)

In [412]:
df_train.iloc[:, 26] = df_train.iloc[:, 26].map(lambda x: 6 if x >= 6 else x)
df_test.iloc[:, 26] = df_test.iloc[:, 26].map(lambda x: 6 if x >= 6 else x)

In [413]:
df_train.iloc[:, 30] = df_train.iloc[:, 30].map(lambda x: 1 if x >= 13 else 0 if not pd.isnull(x) else x)
df_test.iloc[:, 30] = df_test.iloc[:, 30].map(lambda x: 1 if x >= 13 else 0 if not pd.isnull(x) else x)


In [414]:
# TODO High-Cardinality Categorical  Cluster
drop_num.append(35)
# plot(df_train.iloc[:, 35])

In [415]:
df_train['ext_source_2_dummy'] = df_train.iloc[:, 38].map(lambda x: 1 if x> 0.5 else 0 if not pd.isnull(x) else x)
df_test['ext_source_2_dummy'] = df_test.iloc[:, 38].map(lambda x: 1 if x> 0.5 else 0 if not pd.isnull(x) else x)

df_test.iloc[:, 38] = mean_std(df_test.iloc[:, 38])
df_train.iloc[:, 38] = mean_std(df_train.iloc[:, 38])

In [416]:
df_train['ext_source_3_dummy'] = df_train.iloc[:, 39].map(lambda x: 1 if x> 0.42 else 0 if not pd.isnull(x) else x)
df_test['ext_source_3_dummy'] = df_test.iloc[:, 39].map(lambda x: 1 if x> 0.42 else 0 if not pd.isnull(x) else x)

df_test.iloc[:, 39] = mean_std(df_test.iloc[:, 39])
df_train.iloc[:, 39] = mean_std(df_train.iloc[:, 39])

In [417]:
df_train.iloc[:, 40] = df_train.iloc[:, 40].map(lambda x: x if x < 5   else np.nan)
df_test.iloc[:, 40] = df_test.iloc[:, 40].map(lambda x: x if x < 5  else np.nan)

In [418]:
df_train.iloc[:, 41] = df_train.iloc[:, 41].map(lambda x: x if x <= 5 else np.nan)
df_test.iloc[:, 41] = df_test.iloc[:, 41].map(lambda x: x if x <= 5 else np.nan)

In [419]:
df_train.iloc[:, 42] = df_train.iloc[:, 42].map(lambda x: x if x <= 5 else np.nan)
df_test.iloc[:, 42] = df_test.iloc[:, 42].map(lambda x: x if x <= 5 else np.nan)

In [420]:
df_train.iloc[:, 43] = df_train.iloc[:, 43].map(lambda x: x if x <= 2 else 3 if  not pd.isnull(x) else x)
df_test.iloc[:, 43] = df_test.iloc[:, 43].map(lambda x: x if x <= 2 else 3 if  not pd.isnull(x) else x)

In [421]:
df_train.iloc[:, 44] = df_train.iloc[:, 44].map(lambda x: '0' if x < -1000 else '1' if x < 0 else '2' if not pd.isnull(x) else x)
df_test.iloc[:, 44] = df_test.iloc[:, 44].map(lambda x: '0' if x < -1000 else '1' if x < 0 else '2' if not pd.isnull(x) else x)

In [422]:
drop_num.append(45)
drop_num.append(47)
drop_num.append(48)

In [423]:
drop_num.append(53)

In [424]:
drop_num.append(55)

In [425]:
df_train.iloc[:, 65] = df_train.iloc[:, 65].map(lambda x: '0' if x ==0 else '1' if  not pd.isnull(x) else x)
df_test.iloc[:, 65] = df_test.iloc[:, 65].map(lambda x: '0' if x ==0 else '1' if  not pd.isnull(x) else x)

In [426]:
df_train.iloc[:, 66] = df_train.iloc[:, 66].map(lambda x: '0' if x ==0 else '1' if  not pd.isnull(x) else x)
df_test.iloc[:, 66] = df_test.iloc[:, 66].map(lambda x: '0' if x ==0 else '1' if  not pd.isnull(x) else x)

In [427]:
df_train.iloc[:, 67] = df_train.iloc[:, 67].map(lambda x:  '0' if x ==0 else '1' if  not pd.isnull(x) else x)
df_test.iloc[:, 67] = df_test.iloc[:, 67].map(lambda x:  '0' if x ==0 else '1' if  not pd.isnull(x) else x)

In [428]:
df_train.iloc[:, 68] = df_train.iloc[:, 68].map(lambda x:  '0' if x ==0 else '1' if  not pd.isnull(x) else x)
df_test.iloc[:, 68] = df_test.iloc[:, 68].map(lambda x:  '0' if x ==0 else '1' if  not pd.isnull(x) else x)

In [429]:
df_train.iloc[:, 69] = df_train.iloc[:, 69].map(lambda x: 2 if x >= 2 else x)
df_test.iloc[:, 69] = df_test.iloc[:, 69].map(lambda x: 2 if x >= 2 else x)

In [430]:
df_train.iloc[:, 70] = df_train.iloc[:, 70].map(lambda x: 5 if x >= 5 else x)
df_test.iloc[:, 70] = df_test.iloc[:, 70].map(lambda x: 5 if x >= 5 else x)

In [431]:
df_train = df_train.drop(columns=columns[drop_num])
df_test = df_test.drop(columns=columns[drop_num])

In [432]:
df_train['EXT_SOURCE_1'] = ext_1_train
df_test['EXT_SOURCE_1'] = ext_1_test

In [433]:
dropcolum=['FLAG_DOCUMENT_6','FLAG_DOCUMENT_7',
    'FLAG_DOCUMENT_8','FLAG_DOCUMENT_9', 
    'FLAG_DOCUMENT_11','FLAG_DOCUMENT_13',
    'FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16',
    'FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19',
    'FLAG_DOCUMENT_20','FLAG_DOCUMENT_21']
df_train= df_train.drop(dropcolum,axis=1)
df_test= df_test.drop(dropcolum,axis=1)

In [434]:
df_train.index = train_id
df_test.index = test_id

In [454]:
df_train.to_csv('../data/handled/main_train_2.csv')
df_test.to_csv('../data/handled/main_test_2.csv')

# 测试


In [377]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from bayes_opt import BayesianOptimization
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier

/home/ray/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [378]:
x = pd.get_dummies(df_train).values
y = TARGET.values

In [379]:
imp = Imputer()
x = imp.fit_transform(x)

In [437]:
sfm = SelectFromModel(LogisticRegression(penalty='l1', C=0.2))
x = sfm.fit_transform(x, y)

In [439]:
lr = LogisticRegression()
cross_val_score(lr, x, y, scoring='roc_auc')

array([0.74859571, 0.7474503 , 0.75273227])

In [443]:
df_train_dummy = pd.get_dummies(df_train, dummy_na=True)
df_test_dummy = pd.get_dummies(df_test, dummy_na=True)

train, test = df_train_dummy.align(df_test_dummy, join='left', axis=1, fill_value=0)

In [444]:
imp = Imputer()
train = imp.fit_transform(train)
test = imp.transform(test)

In [445]:
sfm = SelectFromModel(LogisticRegression(penalty='l1', C=0.3))
train = sfm.fit_transform(train, y)
test = sfm.transform(test)

In [447]:
lr.fit(train, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [451]:
def output(test_id, test_prob, sid=0):
    result = pd.DataFrame(np.column_stack((test_id, test_prob)))
    result.columns = ['SK_ID_CURR', 'TARGET']
    result['SK_ID_CURR'] = result['SK_ID_CURR'].astype('int')
    result.to_csv('./submission/submission' + str(sid) + '.csv', header=True, index=False)

In [452]:
test_id = df_test.index

In [453]:
pre = lr.predict_proba(test)
output(test_id, pre[:, 1], '_20180707')

# 特征提取、测试

In [21]:
def feature_test(estimator, x, y):
    return cross_val_score(estimator, x, y, cv=5, scoring='roc_auc').mean()

In [22]:
df_train.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,NEW_CAR_TO_EMPLOY_RATIO,NEW_PHONE_TO_BIRTH_RATIO,NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER,NEW_CREDIT_TO_INCOME_RATIO,LOAN_INCOME_RATIO,ANNUITY_INCOME_RATIO,ANNUITY LENGTH,WORKING_LIFE_RATIO,INCOME_PER_FAM,CHILDREN_RATIO
0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,...,NaN,0.991563,NaN,-0.698886,-0.698886,-0.625479,-0.583832,-0.677929,1.108185,-0.626652
1,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,...,NaN,-0.260347,NaN,0.357215,0.357215,-0.517360,1.956586,-0.651455,0.420116,-0.626652
2,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,...,-1.852191,-0.377549,NaN,-0.701880,-0.701880,-0.857556,-0.129157,-1.093932,-0.267952,-0.626652
3,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,...,NaN,-0.560987,NaN,-0.581894,-0.581894,0.408542,-1.345494,0.015787,-0.267952,-0.626652
4,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,...,NaN,-0.151982,NaN,0.141458,0.141458,-0.013178,0.315589,-0.040317,0.282503,-0.626652


In [109]:
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMClassifier

In [43]:
from sklearn.preprocessing import Imputer
im = Imputer()
temp = df_train.loc[:, ['DAYS_BIRTH', 'EXT_SOURCE_2']]
temp = im.fit_transform(temp)

In [44]:
feature_test(GaussianNB(), temp, TARGET)

0.6559337067313342

In [116]:
cls = LGBMClassifier(
nthread=4,
            # is_unbalance=True,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=32,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.04,
            reg_lambda=0.073,
            min_split_gain=0.0222415,
            min_child_weight=40,
            silent=-1,
            verbose=-1)

In [123]:
df_train['a1'] = df_train.DAYS_BIRTH * df_train.EXT_SOURCE_2
df_train['a2'] = df_train.DAYS_BIRTH * df_train.EXT_SOURCE_2 * df_train.EXT_SOURCE_3
df_train['a3'] = df_train.DAYS_BIRTH / df_train.EXT_SOURCE_2

In [131]:
temp = pd.get_dummies(df_train.loc[:, ['a3', 'DAYS_BIRTH', 'EXT_SOURCE_2',
                                       'EXT_SOURCE_3', 'ORGANIZATION_TYPE', 'HOUR_APPR_PROCESS_START', 'AMT_ANNUITY']])
feature_test(LGBMClassifier(), temp, TARGET)

0.723385497343698

In [52]:
df_train['TARGET'] = TARGET

In [53]:
df_train['TARGET']= df_train.TARGET.astype('int')

In [54]:
## 对 ORGANIZATION_TYPE聚类

res = df_train.groupby(by='ORGANIZATION_TYPE')['TARGET'].mean()

In [55]:
res = pd.DataFrame(res)

In [56]:
from sklearn.cluster import KMeans

In [90]:
kmeans = KMeans(n_clusters=4).fit(res)

In [91]:
kmeans.labels_

array([2, 3, 1, 2, 2, 3, 3, 3, 1, 2, 2, 2, 2, 2, 3, 2, 2, 1, 0, 2, 3, 3,
       2, 2, 2, 0, 2, 1, 2, 2, 2, 1, 3, 2, 1, 2, 3, 1, 3, 1, 3, 1, 3, 2,
       2, 3, 2, 3, 1, 1, 1, 3, 1, 2, 0, 3, 1, 1], dtype=int32)

In [92]:
map_dict = dict(zip(res.index, kmeans.labels_))

In [100]:
df_train['ORGANIZATION_CLUSTER'] = df_train.ORGANIZATION_TYPE.map(map_dict)
df_test['ORGANIZATION_CLUSTER'] = df_test.ORGANIZATION_TYPE.map(map_dict)

In [94]:
temp = pd.get_dummies(df_train.loc[:, ['ORGANIZATION_CLUSTER']])
feature_test(GaussianNB(), temp, TARGET)

0.5627692813325408

In [101]:
df_train.index = train_id
df_test.index = test_id

In [106]:
pd.concat((df_train.loc[:, 'ORGANIZATION_CLUSTER'], df_test.loc[:, 'ORGANIZATION_CLUSTER'])).to_csv("../../data/handled/new_features/orgnz.csv")

In [108]:
df_train.ORGANIZATION_TYPE.isnull().sum()

0